#### Import the Python dependencies

In [3]:
import urllib
from _settings import * #this has some sensitive stuff like api keys and usernames for carto
import os, sys
import pandas as pd
import subprocess

import warnings
warnings.filterwarnings('ignore')

#### Create the data folder structure

In [4]:
pathList = ['input','output','processing','machine_learning','machine_learning/output']

dataDir = 'data_v2/'

def makeDirectory(path):
    try:
        os.makedirs(path)
    except:
        pass
        
for i in pathList:
    makeDirectory(dataDir + i)

In [54]:
SAVE_FOLDER = dataDir
INPUT_FOLDER = SAVE_FOLDER + 'input/'
PROCESSING_FOLDER = SAVE_FOLDER + 'processing/'
ML_FOLDER = SAVE_FOLDER + 'machine_learning/'
OUTPUT_FOLDER = SAVE_FOLDER + 'output/'
curDir = os.getcwd()
print curDir

/Users/danielmsheehan/GitHub/osm_walkability


#### Stop here if just trying to import modules and set the directories. 

## Download the input data

In [9]:
url = 'https://dms2203.cartodb.com/api/v2/sql?filename=beh_walkability_ct2010.csv&format=csv&q=SELECT geoid,t10km2,t10lndkm2,t10cnt,t10resdn1,t10intden,t10entrpy,t10rtlfar,t10sub07d,t10walk,t10walkc,the_geom FROM ct10'

dl_file = urllib.URLopener()
dl_file.retrieve(url, INPUT_FOLDER + 'beh_nyc_walkability.csv')

url = 'https://s3.amazonaws.com/metro-extracts.mapzen.com/new-york_new-york.osm.pbf'

dl_file = urllib.URLopener()
dl_file.retrieve(url, INPUT_FOLDER + 'new-york_new-york.osm.pbf')

('data_v2/input/beh_nyc_walkability.csv',
 <httplib.HTTPMessage instance at 0x10ee5ddd0>)

## Paste GDAL/OGR commands into terminal
Currently not working using os.system in this virtual env

In [5]:
thecmd = 'ogr2ogr -f "SQLite" -dsco SPATIALITE=YES ' + dataDir + 'input/new-york_new-york.db ' + dataDir + 'input/new-york_new-york.osm.pbf'
print thecmd

ogr2ogr -f "SQLite" -dsco SPATIALITE=YES data_v2/input/new-york_new-york.db data_v2/input/new-york_new-york.osm.pbf


In [16]:
thecmd = 'ogr2ogr -f "CSV" ' + dataDir + 'input/new-york_new-york_points.csv ' + dataDir + 'input/new-york_new-york.db -lco GEOMETRY=AS_XY -progress -explodecollections -sql "select * from points"'
print thecmd

ogr2ogr -f "CSV" data_v2/input/new-york_new-york_points.csv data_v2/input/new-york_new-york.db -lco GEOMETRY=AS_XY -progress -explodecollections -sql "select * from points"


## Clean up the OSM data
So its just **osm_id** and **latitude, longitude**

In [6]:
inCSV = dataDir+'input/new-york_new-york_points.csv'
df = pd.read_csv(inCSV)
df = df[['X','Y','osm_id']]
df.columns = ['longitude','latitude','osm_id'] #just delcare col names here
df.to_csv(dataDir+'processing/new-york_new-york_points.csv', index=False)

## Upload OSM point data to Carto 

In [7]:
thecmd = 'curl -v -F file=@'+curDir+'/'+dataDir+'processing/new-york_new-york_points.csv "https://'+USERNAME+'.carto.com/api/v1/imports/?api_key="'+APIKEY
os.system(thecmd) #run the command to curl the input file, this should work as its not GDAL/OGR

0

## Upload the BEH Walkability data to Carto

In [10]:
thecmd = 'curl -v -F file=@'+curDir+'/'+dataDir+'input/beh_nyc_walkability.csv "https://'+USERNAME+'.carto.com/api/v1/imports/?api_key="'+APIKEY
os.system(thecmd) #run the command to curl the input file, this should work as its not GDAL/OGR

0

## Run an intersection to via the Carto Batch SQL API

In [11]:
query = 'CREATE TABLE new_york_new_york_points_int_ct10 AS (SELECT geoid, osm_id, latitude, longitude FROM beh_nyc_walkability, new_york_new_york_points WHERE ST_INTERSECTS(beh_nyc_walkability.the_geom, new_york_new_york_points.the_geom))'
thecmd = '''curl -X POST -H "Content-Type: application/json" -d '{
  "query": "'''+query+'''"
}' "http://'''+USERNAME+'''.carto.com/api/v2/sql/job?api_key='''+APIKEY+'''"'''

0

In [11]:
batcmd = thecmd
result = subprocess.check_output(batcmd, shell=True)

In [29]:
print result

{"user":"dms2203","query":"SELECT cdb_cartodbfytable(new_york_new_york_points_int_ct10_v2)","job_id":"43d3b877-3ba0-4273-829b-beb1cf5af9a3","created_at":"2017-09-07T21:22:59.866Z","updated_at":"2017-09-07T21:22:59.866Z","status":"pending"}
curl -X POST -H "Content-Type: application/json" -d '{
  "query": "SELECT cdb_cartodbfytable('new_york_new_york_points_int_ct10_v2')"
}' "http://dms2203.carto.com/api/v2/sql/job?api_key=eb4e90135e42f41bda36e58b7e9c1a660acd98d1"


## CartoDBfy the resulting table

In [12]:
query = "SELECT cdb_cartodbfytable('new_york_new_york_points_int_ct10')"
thecmd = '''curl -X POST -H "Content-Type: application/json" -d '{
  "query": "'''+query+'''"
}' "http://'''+USERNAME+'''.carto.com/api/v2/sql/job?api_key='''+APIKEY+'''"'''

In [13]:
batcmd = thecmd
result = subprocess.check_output(batcmd, shell=True)

In [14]:
print result

{"user":"dms2203","query":"SELECT cdb_cartodbfytable(new_york_new_york_points_int_ct10)","job_id":"d2871209-b5f8-4f1c-9e2a-cb05a737db9d","created_at":"2017-09-11T15:27:00.351Z","updated_at":"2017-09-11T15:27:00.351Z","status":"pending"}


**Or run your SQL via the [Carto Batch API](https://cartodb.github.io/customer_success/batch/)**

## Download the OSM - BEH Walkability Census Tracts 

In [15]:
url = 'https://dms2203.cartodb.com/api/v2/sql?format=csv&q=SELECT geoid, osm_id, latitude, longitude FROM new_york_new_york_points_int_ct10&api_key='+APIKEY

dl_file = urllib.URLopener()
dl_file.retrieve(url, PROCESSING_FOLDER+'/new_york_new_york_points_int_ct10.csv')

('data_v2/processing//new_york_new_york_points_int_ct10.csv',
 <httplib.HTTPMessage instance at 0x114b5d4d0>)

## Split up OSM Highway & Tags Other into seperate files

In [16]:
inCSV = INPUT_FOLDER+'/new-york_new-york_points.csv'

df = pd.read_csv(inCSV)

dfH = df[['osm_id','highway']]
dfT = df[['osm_id','other_tags']]

dfH = dfH.dropna(subset=['highway'])

ouCSV = PROCESSING_FOLDER+'/osm_highway.csv'
dfH.to_csv(ouCSV, index=False)

dfT = dfT.dropna(subset=['other_tags'])

ouCSV = PROCESSING_FOLDER+'/osm_other_tags.csv'
dfT.to_csv(ouCSV, index=False)

## Highway Pivot

In [17]:
import pandas as pd
import numpy as np

inCSV = PROCESSING_FOLDER+'/osm_highway.csv'

df = pd.read_csv(inCSV)

df['count'] = 1

dfp = pd.pivot_table(df, values='count', index='osm_id',columns='highway', aggfunc=np.sum)

dfp = dfp.rename(columns=lambda x: x.replace(';', '_'))

ouCSV = PROCESSING_FOLDER+'/osm_highway_pivot.csv'

dfp.to_csv(ouCSV)

## Parse Tags Other 
This took a very long long time to run. I should look into rewriting the code, maybe not store all these dataframes into memory.

In [18]:
inCSV = PROCESSING_FOLDER + '/osm_other_tags.csv'

df = pd.read_csv(inCSV)

tagList = df.values.tolist()

df_list = []

for h in tagList:
    bucket = []
    x = h[1].split('","')
    
    for i in x:
        z = i.split("=>")
        osm = []
        
        for j in z: 
            osm.append(j.replace('"','').replace(':','_'))
        bucket.append(osm)

    df = pd.DataFrame(data=[h[0]], columns=['osm_id'])
    
    for i in bucket:
        df[i[0]] = i[1]

    df_list.append(df)

print '...concatenating...'
df = pd.concat(df_list)
df = df.set_index('osm_id')

df.to_csv(PROCESSING_FOLDER + '/osm_tags_pivot.csv')

...concatenating...


## Clean up Other Tags reduce low features

In [21]:
inCSV = PROCESSING_FOLDER + '/osm_tags_pivot.csv'

df = pd.read_csv(inCSV)

dfLen = len(df.index)

keeperCols = []

for i in df.columns:
    countNulls = df[i].isnull().sum()
    pctNull = countNulls/(dfLen * 1.0)
    if df[i].nunique() < 500:
        if pctNull < 0.99:
            print i, pctNull, df[i].nunique()
            keeperCols.append(i)

keeperCols     
    
df = df[keeperCols+['osm_id']]        

ouCSV = PROCESSING_FOLDER + '/osm_tags_pivot_cln.csv'

df.to_csv(ouCSV, index=False)

addr_city 0.968342437816 363
addr_state 0.971963337785 16
amenity 0.880495836688 122
capacity 0.975261201406 88
cityracks.large 0.977893706577 9
cityracks.small 0.977893706577 9
crossing 0.974528342824 10
cuisine 0.985419971361 374
gnis_county_id 0.958183667861 29
gnis_created 0.956901165342 169
gnis_state_id 0.958183667861 3
leisure 0.988780513676 35
natural 0.966071540498 15
operator 0.988370691442 404
power 0.953612944597 10
railway 0.968294223435 21
religion 0.986027472554 9
shop 0.977127097928 183


## Merge data, One Hot Encode, Normalize variables to Count by Square Kilometer

In [22]:
inTra = INPUT_FOLDER + '/beh_nyc_walkability.csv'
inInt = PROCESSING_FOLDER + '/new_york_new_york_points_int_ct10.csv'
inHig = PROCESSING_FOLDER + '/osm_highway_pivot.csv'
inTag = PROCESSING_FOLDER + '/osm_tags_pivot_cln.csv'

dfInt = pd.read_csv(inInt)
dfTra = pd.read_csv(inTra)
dfHig = pd.read_csv(inHig)
dfTag = pd.read_csv(inTag)

dfInt = dfInt[['geoid','osm_id']]
dfTra = dfTra[['geoid','t10walk','t10lndkm2']]

df = dfInt.merge(dfTra, how='left', on='geoid')
df = df.merge(dfHig, how='left', on='osm_id')
df = df.merge(dfTag, how='left', on='osm_id')

df = df.drop(['osm_id','gnis_created','gnis_county_id','gnis_state_id','capacity','addr_state','addr_city','religion'], axis=1)
#df = df.drop(['osm_id'], axis=1)

dfStringCols = []

for i in df.columns:
    if df[i].dtype == 'object': #select all non numerical columns
        dfStringCols.append(i)

df['index_col'] = df.index

colTotal = 0
for i in dfStringCols:
    one_hot = pd.get_dummies(df[i])
    colTotal = colTotal + len(one_hot.columns)
    one_hot = one_hot.rename(columns = lambda x : i + '_one_hot_' + x)
    one_hot['index_col'] = one_hot.index
    df = df.merge(one_hot, on='index_col', how='left')

df = df.drop(dfStringCols, axis=1)
df = df.groupby(['geoid','t10walk','t10lndkm2'],as_index=False).sum() # sum osm features to tract boundaries
df = df.fillna(0)

colsList = df.columns

for i in colsList:
    if i != 'geoid' and i != 'osm_id'  and i != 't10walk'  and i != 't10lndkm2':
        df[i] = df[i]/df['t10lndkm2'] # get osm feature density by tract area

df = df.drop(['t10lndkm2','index_col'], axis=1)
df = df.fillna(0)

ouCSV = PROCESSING_FOLDER+'/walkability_highway_tags_features.csv'
df.to_csv(ouCSV, index=False)

## Create Training and Test datasets

In [23]:
inCSV = PROCESSING_FOLDER + '/walkability_highway_tags_features.csv'

df = pd.read_csv(inCSV)

msk = np.random.rand(len(df)) < 0.8 #split 80/20, 80% for training, 20% for testing - http://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas
train = df[msk]
test = df[~msk]

print len(df)
print len(train)
print len(test)

test.to_csv(ML_FOLDER + '/test_w_label.csv', index=False)
df.to_csv(ML_FOLDER + '/all_w_label.csv', index=False)

testDropCols = ['t10walk'] #['health','label']

test = test.drop(testDropCols,axis=1)
df = df.drop(testDropCols,axis=1)

train.to_csv(ML_FOLDER + '/train.csv',index=False)
test.to_csv(ML_FOLDER + '/test.csv', index=False)
df.to_csv(ML_FOLDER + '/all.csv', index=False)

2163
1764
399


## Machine Learning

In [24]:
from sklearn.cross_validation import cross_val_score
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from datetime import datetime
from sklearn import metrics
from sklearn import tree

print datetime.now()
 
wd = ML_FOLDER + '/'
 
def main():
    data = pd.read_csv(wd + 'train.csv').astype(np.float32)
    data = data.replace(np.inf, 0)
    data = data.drop('geoid',axis=1)
    data = data.fillna(data.mean())
    feature_names = data.columns
    
    t0 = datetime.now()

    X = data[data.columns.difference(['t10walk'])]
    Y = data['t10walk']

    clf = tree.DecisionTreeRegressor()
    #clf = ExtraTreeRegressor()

    print datetime.now()
 
    t0 = datetime.now()
    clf.fit(X, Y)
    print "training time:", (datetime.now() - t0)
    print "training accuracy:", clf.score(X, Y)
 
    quiz = pd.read_csv(wd + 'all.csv').astype(np.float32)
    quiz = quiz.drop('geoid',axis=1)
    quiz = quiz.replace(np.inf, 0)

    t0 = datetime.now()

    Xtest = quiz[quiz.columns.difference(['t10walk'])]

    prediction = clf.predict(Xtest) # do prediction and save it

    print 'cross validating...'
    scores = cross_val_score(clf, X, Y, cv=5)
    print scores
    
    ftime = str(datetime.now()).replace(' ','-').replace(':','-').replace('.','-')
    print ftime
 
    ouCSV = wd+'output/myoutput-'+ftime+'.csv'

    with open(ouCSV, 'w') as f:
        f.write('id,prediction\n')
        the_id = 1
        for p in prediction:
            f.write("%s,%s\n" % (the_id, p))
            the_id += 1
 
    importance = pd.DataFrame(zip(data.columns.difference(['t10walk']), clf.feature_importances_) )
    importance.columns = ['feature', 'importance']
    importance = importance.sort_values('importance',ascending=False)
    importance.to_csv(wd+'output/myoutput-'+ftime+'_importances.csv', index=False)  
    
    features_names = feature_names[1:]

    tree.export_graphviz(clf, out_file='tree.dot', feature_names=feature_names[1:], filled=True, leaves_parallel=True)
 
if __name__ == '__main__':
    main()

/Users/danielmsheehan/GitHub/osm_walkability/env/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


2017-09-11 13:21:53.771493
2017-09-11 13:21:54.002022
training time: 0:00:00.033346
training accuracy: 0.937870370514
cross validating...
[-0.2219347  -2.36244634 -0.01105474 -0.80244384 -0.63520043]
2017-09-11-13-21-54-361501


## Rejoin with input data

In [56]:
df = pd.read_csv(wd + 'all_w_label.csv')

pred = pd.read_csv(wd + 'output/myoutput-2017-09-11-13-21-54-361501.csv')
geog = pd.read_csv(INPUT_FOLDER + 'beh_nyc_walkability.csv')

df['index_col'] = df.index
pred['index_col'] = pred.index

In [57]:
df = df[['geoid','index_col','t10walk']]

df = df.merge(pred, on='index_col', how='left').merge(geog, on='geoid', how='left')

df.to_csv(OUTPUT_FOLDER + 'beh_nyc_walkability_prediction.csv', index=False)

##  Prediction joined to Walkability

In [59]:
df.head()

,geoid,index_col,t10walk_x,id,prediction,t10km2,t10lndkm2,t10cnt,t10resdn1,t10intden,t10entrpy,t10rtlfar,t10sub07d,t10walk_y,t10walkc,the_geom
0,36005000100,0,0.0000,1,0.0000,1.691295,1.690813,25,-4.5421,-2.9774,0.0000,-2.2935,-0.6993,0.0000,0,0106000020E610000001000000010300000001000000F9...
1,36005000200,1,-2.9096,2,-2.9096,0.476454,0.473260,46,-0.0690,-0.4720,-1.3325,-0.3367,-0.6993,-2.9096,1,0106000020E6100000010000000103000000010000009D...
2,36005000400,2,-2.7511,3,-2.7511,0.799501,0.799500,37,-0.1032,-0.5378,-1.1057,-0.3050,-0.6993,-2.7511,1,0106000020E61000000200000001030000000100000020...
3,36005001600,3,-2.2716,4,-2.2716,0.485080,0.485080,20,0.1547,-1.3235,0.2939,-0.6975,-0.6993,-2.2716,2,0106000020E6100000010000000103000000010000001C...
4,36005001900,4,0.2115,5,0.2115,1.692264,1.692264,92,0.3584,-0.8116,0.2951,-0.0924,0.3400,0.2115,3,0106000020E6100000030000000103000000010000000C...


## Map of Predicted Walkability and BEH Walkability 1.0

In [64]:
from IPython.display import IFrame
IFrame('https://dms2203.carto.com/builder/e18a2cd4-0edb-412b-9b40-36b08216fe5f/embed?state=%7B%22map%22%3A%7B%22ne%22%3A%5B40.34445080136368%2C-74.66445922851564%5D%2C%22sw%22%3A%5B41.05760862509861%2C-73.35433959960939%5D%2C%22center%22%3A%5B40.702244436175114%2C-74.01008605957033%5D%2C%22zoom%22%3A10%7D%2C%22widgets%22%3A%7B%2257f9458a-b853-444f-a937-0c2bbb5738a6%22%3A%7B%22autoStyle%22%3Atrue%2C%22normalized%22%3Atrue%7D%2C%2210db2f20-bdb2-42dd-a518-cfbb0a34c211%22%3A%7B%22normalized%22%3Atrue%7D%7D%7D', width='100%', height=700)